<a href="https://colab.research.google.com/github/Ellinei/229352-StatisticalLearning/blob/main/Lab03_kNN_using_Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #3

In [29]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


In [30]:
print("X[0]:", Xtrain[0])
print("y[0]:", ytrain[0])

X[0]: From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





y[0]: 7


In [31]:
train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Apply Tfidf ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html))

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

# define transformation
tfidf = TfidfVectorizer()

# fit+transform training set
Xtrain_tfidf = tfidf.fit_transform(Xtrain)

# See output
Xtrain_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 470232 stored elements and shape (3000, 61994)>

In [33]:
#tfidf.vocabulary_

#### Exercise 1: Find post in the training set that is closest in tf-idf to the first post in the test set (`Xtest[0]`). Print the content of both posts (not the tf-idf vectors).

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

Xtest_tfidf = tfidf.transform(Xtest)
similarity_scores = cosine_similarity(Xtest_tfidf[0], Xtrain_tfidf)
closest_index = similarity_scores.argmax()
print("First test post:")
print(Xtest[0])
print("Closest training post:")
Xtrain[closest_index]

First test post:
From: v064mb9k@ubvmsd.cc.buffalo.edu (NEIL B. GANDLER)
Subject: Need info on 88-89 Bonneville
Organization: University at Buffalo
Lines: 10
News-Software: VAX/VMS VNEWS 1.41
Nntp-Posting-Host: ubvmsd.cc.buffalo.edu


 I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.

			Neil Gandler

Closest training post:


'From: v063kcbp@ubvmsd.cc.buffalo.edu (MITCH)\nSubject: Thanks! (Backing Up Masters)\nKeywords: Misled, Confused, Advice, Comprendo!\nOrganization: University at Buffalo\nLines: 8\nNews-Software: VAX/VMS VNEWS 1.41\nNntp-Posting-Host: ubvmsd.cc.buffalo.edu\n\n\n\tJust a quick THANKS to the many who explained the backing up of my\nmasters.  Apparently they are NOT copy-protected; I just used a program that\nis unable to handle high-density (old shit).  I was surprised to hear that\n"NO programs on high-density disks have copy protection," which someone\nback there said.  Huh!  Learn something new every day!\n\n\t\t\t\t\t\t- Mitch\n'

### Classify with k-Nearest Neighbor (kNN) ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html))

In [35]:
from sklearn.neighbors import KNeighborsClassifier

# Define model
nb = KNeighborsClassifier()

# Fit the model
nb.fit(Xtrain_tfidf, ytrain)

KNeighborsClassifier()

Evaluate on the test set using [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

We will focus on the [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [36]:
from sklearn.metrics import classification_report

# Transform the test set
Xtest_tfidf = tfidf.transform(Xtest)

# Make predictions on the test set
ypred = nb.predict(Xtest_tfidf)

# report classification scores
print(classification_report(ytest, ypred,
                            target_names=train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.18      0.48      0.26        21
           comp.graphics       0.38      0.48      0.43        21
 comp.os.ms-windows.misc       0.56      0.58      0.57        26
comp.sys.ibm.pc.hardware       0.52      0.50      0.51        34
   comp.sys.mac.hardware       0.67      0.53      0.59        34
          comp.windows.x       0.88      0.54      0.67        26
            misc.forsale       0.50      0.41      0.45        22
               rec.autos       0.58      0.54      0.56        28
         rec.motorcycles       0.74      0.70      0.72        33
      rec.sport.baseball       0.82      0.72      0.77        25
        rec.sport.hockey       0.75      0.56      0.64        27
               sci.crypt       0.71      0.85      0.77        20
         sci.electronics       0.31      0.21      0.25        24
                 sci.med       0.79      0.48      0.59        23
         

### Combine all methods into a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Define pipeline with TfidfVectorizer and KNeighborsClassifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knn', KNeighborsClassifier())
])

# Fit the pipeline to the training set
pipeline.fit(Xtrain, ytrain)

# Make predictions on the test set
test_pred = pipeline.predict(Xtest)

# report classification scores
print(classification_report(ytest, test_pred,
                            target_names=train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.18      0.48      0.26        21
           comp.graphics       0.38      0.48      0.43        21
 comp.os.ms-windows.misc       0.56      0.58      0.57        26
comp.sys.ibm.pc.hardware       0.52      0.50      0.51        34
   comp.sys.mac.hardware       0.67      0.53      0.59        34
          comp.windows.x       0.88      0.54      0.67        26
            misc.forsale       0.50      0.41      0.45        22
               rec.autos       0.58      0.54      0.56        28
         rec.motorcycles       0.74      0.70      0.72        33
      rec.sport.baseball       0.82      0.72      0.77        25
        rec.sport.hockey       0.75      0.56      0.64        27
               sci.crypt       0.71      0.85      0.77        20
         sci.electronics       0.31      0.21      0.25        24
                 sci.med       0.79      0.48      0.59        23
         

Now we will use [grid search cross-validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find model with the best hyperparameters

![5CV](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [38]:
from sklearn.model_selection import GridSearchCV

params = {}

# Define GridSearchCV
gridcv = GridSearchCV(pipeline, params,
                      scoring='f1_macro', cv=3)

# Fit and cross-validate the model on 3-fold data
gridcv.fit(Xtrain, ytrain)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={}, scoring='f1_macro')

In [39]:
gridcv.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('knn', KNeighborsClassifier())])

In [40]:
# Make predictions on the test set
test_pred = gridcv.predict(Xtest)

# report classification scores
print(classification_report(ytest, test_pred,
                            target_names=train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.18      0.48      0.26        21
           comp.graphics       0.38      0.48      0.43        21
 comp.os.ms-windows.misc       0.56      0.58      0.57        26
comp.sys.ibm.pc.hardware       0.52      0.50      0.51        34
   comp.sys.mac.hardware       0.67      0.53      0.59        34
          comp.windows.x       0.88      0.54      0.67        26
            misc.forsale       0.50      0.41      0.45        22
               rec.autos       0.58      0.54      0.56        28
         rec.motorcycles       0.74      0.70      0.72        33
      rec.sport.baseball       0.82      0.72      0.77        25
        rec.sport.hockey       0.75      0.56      0.64        27
               sci.crypt       0.71      0.85      0.77        20
         sci.electronics       0.31      0.21      0.25        24
                 sci.med       0.79      0.48      0.59        23
         

#### Exercise 2:

1. Use grid search 5-fold cross-validation across different values of the following two kNN parameters: `n_neighbors` and `metric`  **on the training set** to find the best model.

2. For the best value of `n_neighbors` and `metric` you found above, compute the `f1_macro` score **on the test set**.
* Print the value of `n_neighbors` and `metric`.
* Print the model's `f1_macro` score.

In [41]:
params = {
    'knn__n_neighbors': [3, 5, 7, 9],
    'knn__metric': ['euclidean', 'cosine']
}
gridcv = GridSearchCV(pipeline, params, scoring='f1_macro', cv=5)
gridcv.fit(Xtrain, ytrain)
print("Best parameters: ", gridcv.best_params_)
test_pred = gridcv.predict(Xtest)
print(classification_report(ytest, test_pred, target_names=train.target_names))

Best parameters:  {'knn__metric': 'euclidean', 'knn__n_neighbors': 5}
                          precision    recall  f1-score   support

             alt.atheism       0.18      0.48      0.26        21
           comp.graphics       0.38      0.48      0.43        21
 comp.os.ms-windows.misc       0.56      0.58      0.57        26
comp.sys.ibm.pc.hardware       0.52      0.50      0.51        34
   comp.sys.mac.hardware       0.67      0.53      0.59        34
          comp.windows.x       0.88      0.54      0.67        26
            misc.forsale       0.50      0.41      0.45        22
               rec.autos       0.58      0.54      0.56        28
         rec.motorcycles       0.74      0.70      0.72        33
      rec.sport.baseball       0.82      0.72      0.77        25
        rec.sport.hockey       0.75      0.56      0.64        27
               sci.crypt       0.71      0.85      0.77        20
         sci.electronics       0.31      0.21      0.25        24
     

In [42]:
from sklearn.metrics import f1_score

best_n_neighbors = gridcv.best_params_['knn__n_neighbors']
best_metric = gridcv.best_params_['knn__metric']

print(f"Best n_neighbors: {best_n_neighbors}")
print(f"Best metric: {best_metric}")

test_pred = gridcv.predict(Xtest)
f1_macro = f1_score(ytest, test_pred, average='macro')
print(f"F1-macro score: {f1_macro}")

Best n_neighbors: 5
Best metric: euclidean
F1-macro score: 0.5423793486154647
